### Setup

In [ ]:
import timm
import torch
import numpy as np
from PIL import Image
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import mplcursors
%matplotlib widget

### Model Loading

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = timm.create_model(
    'vit_large_patch16_dinov3.lvd1689m',
    pretrained=True,
    num_classes=0,
)

model = model.to(device)
model = model.eval()

### Loading Images

In [ ]:
IMAGES_ROOT = './images/ball'

dataset = timm.data.create_dataset(
    name='',
    root=IMAGES_ROOT,
    split='',
)

data_config = timm.data.resolve_model_data_config(model)

loader = timm.data.create_loader(
    dataset,
    batch_size=1,
    pin_memory=(device == 'cuda'),
    is_training=False,
    **data_config
)

### Embeddings Extraction

In [ ]:
embeddings = []

with torch.no_grad():
    for batch, _ in loader:
        batch = batch.to(device)
        emb = model(batch)
        embeddings.append(emb.cpu())

embeddings = torch.cat(embeddings, dim=0)

embeddings_np = embeddings.numpy()

scaler = StandardScaler()
embeddings_scaled = scaler.fit_transform(embeddings_np)

### Embeddings Analysis

#### Difference between embedding components

In [ ]:
for idx1, img1 in enumerate(dataset.filenames()):
    for idx2, img2 in enumerate(dataset.filenames()):
        if (idx1 >= idx2): continue
        diff = embeddings_np[idx1] - embeddings_np[idx2]
        dims_changed = np.where(np.abs(diff) > 0.10)[0]
        print(f'Comparing {img1}, {img2}:')
        print(f'\tChanged dims:', dims_changed.tolist(), end='\n\n')

#### Cosine similiraty

In [ ]:
COLOR1 = 'blue'
COLOR2 = 'white'

transforms = timm.data.create_transform(**data_config, is_training=False)

def get_embedding(img_path):
    img = Image.open(img_path).convert('RGB')
    tensor = transforms(img).unsqueeze(0).to(device)
    with torch.no_grad():
        emb = model(tensor).cpu().squeeze(0).numpy()
    return emb

ball_color1_emb = get_embedding(f'./images/ball/ball_{COLOR1}.png')
ball_color2_emb = get_embedding(f'./images/ball/ball_{COLOR2}.png')
chair_color1_emb = get_embedding(f'./images/chair/chair_{COLOR1}.png')
chair_color2_emb = get_embedding(f'./images/chair/chair_{COLOR2}.png')

cos_sim = cosine_similarity([ball_color1_emb - ball_color2_emb], [chair_color1_emb - chair_color2_emb])[0, 0]
print(cos_sim)
cos_sim = cosine_similarity([ball_color1_emb - chair_color1_emb], [ball_color2_emb - chair_color2_emb])[0, 0]
print(cos_sim)

### PCA

In [ ]:
# 2D PCA
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings_scaled)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], s=100)
plt.xlabel(f'PC1 {pca.explained_variance_ratio_[0]:.2%} variance')
plt.ylabel(f'PC2 {pca.explained_variance_ratio_[1]:.2%} variance')
plt.title('DINOv3 Embeddings - PCA')
plt.grid(True, alpha=0.3)

cursor = mplcursors.cursor(scatter, hover=True)
@cursor.connect('add')
def on_add(sel):
    sel.annotation.set_text(dataset.filenames()[sel.index])

plt.show()

print(f"Explained variance (first 2 components): {pca.explained_variance_ratio_[:2].sum():.2%}")

### t-SNE

In [ ]:
tsne = TSNE(n_components=2, random_state=42, perplexity=5)
embeddings_tsne = tsne.fit_transform(embeddings_scaled)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(embeddings_tsne[:, 0], embeddings_tsne[:, 1], s=100)
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.title('DINOv3 Embeddings - t-SNE')
plt.grid(True, alpha=0.3)

cursor = mplcursors.cursor(scatter, hover=True)
@cursor.connect('add')
def on_add(sel):
    sel.annotation.set_text(dataset.filenames()[sel.index])

plt.show()